## BEP code verificatie voor simplificaties van de radiatieve functies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.optimize import curve_fit

In [ ]:
h = 6.62*10**-34 #plancks constant
kb = 1.38*10**-23 #boltzmanns constant
c = 3*10**8 #speed of light

def rc_emissivity(labda):
    if labda <= 2.5e-6:
        e = 0.041
    elif labda > 2.5e-6 and labda <= 8e-6:
        e = 0.4
    elif labda > 8e-6:
        e = 0.85
    return e
def atm_emissivity(labda):
    e_rc = rc_emissivity(labda)
    if labda >= 8e-6 and labda <= 13e-6:
        e = 0.8
    else:
        e = 1
    return e*e_rc

def Planck(labda,T,emissivity_func):
    e = 1
    return 2*h*c**2/(labda**5 *np.expm1(h*c/(labda*kb*T)))*np.pi*e




Define different radiative functions:

In [ ]:
T = np.linspace(253,297, 100)

def integrate_planck(Planck,T,emissivity):
    Ilist = []
    for temperature in T:
        I = quad(Planck,0.5e-6,5e-5, args=(temperature,emissivity), limit=200)
        Ilist.append(I[0])
    return Ilist

Irc = integrate_planck(Planck,T,rc_emissivity)
Iatm = integrate_planck(Planck,T,atm_emissivity)
print(Irc)



plot values + first order taylor expansion

In [ ]:
def f(x,a,b):
    return a*x+b
popt,pcov = curve_fit(f,T,Irc)
popt1,pcov1 = curve_fit(f,T,Iatm)
x = np.linspace(T[0],T[-1],1000)
plt.figure()
plt.title('Radiative power of a blackbody radiator between -20 and 25 deg C at thermal infrared')
plt.plot(T,Irc,'r.', label = 'Numerically integrated radiative power')
plt.plot(x,f(x,*popt1), color = 'black',label = 'Linear fit')
plt.xlabel('Temperature (K)')
plt.ylabel('Radiance (W/m^2)')
plt.legend()
plt.show()
print(*popt, *popt1)
print('standard deviation is:', np.sqrt(np.diag(pcov)[0]))

In [ ]:
print(np.max(Irc-f(T,*popt1)))   #print the maximum error in these functions